In [1]:
import numpy as np
import cvxpy as cp
from scipy.io import loadmat

Load $Y_{bus}$ matrix from MATLAB

In [2]:
# Note: in per-unit
Y = loadmat('Ybus')['Ybus'].todense()

Problem parameters

In [3]:
# Number of buses
N = Y.shape[0]

# Susceptance matrix. Recall in DC power flow, G = 0 and shunt admittances are zero.
B = np.array(np.imag(Y))
B = B - np.diag(np.diag(B))
B = B - np.diag(np.sum(B, axis=0))

# System base, in MVA
base = 100

# The load consumed at each bus
p_d = np.array([40, 150, 80, 130])/base

# Matrix defining the cost of the generators
# The i, j element is the coefficient of the power produced at the ith generator raised to the j-1 power
# Copied from PowerWorld
C = np.array(
    [
        [373.5, 10, 0.016],
        [403.6, 8, 0.018],
        [253.2, 12, 0.018]
    ]
)

# Line constraints. The i, j element is the MVA limit of the line from bus i to bus j
P_line = np.array(
    [
        [0, 150, 150, 0, 0],
        [0, 0, 120, 100, 200],
        [0, 0, 0, 222, 0],
        [0, 0, 0, 0, 60],
        [0, 0, 0, 0, 0]
    ]
)
P_line = (P_line+P_line.T)/base

# Minimum and maximum generator outputs, taken from PowerWorld
p_min = np.array([100, 150, 0])/base
p_max = np.array([400, 500, 300])/base

# The buses with generators and loads
G = np.array([1, 2, 4])-1
L = np.array([2, 3, 4, 5])-1

# Transformation matrices. These handle the change-of-variables from the load/generator spaces to the bus space.
AG = np.zeros((N,len(G)))
AG[G,:] = np.eye(len(G))
AL = np.zeros((N,len(L)))
AL[L,:] = np.eye(len(L))

First, we solve conventional supply-side OPF. Confirm that the total cost is consistent with PowerWorld.

In [4]:
p_g = cp.Variable(len(G))
delta = cp.Variable(N)
constraints = [
    p_g >= p_min,
    p_g <= p_max,
    AG@p_g-AL@p_d == -B@delta,
    cp.multiply(B, delta[:,np.newaxis]-delta[np.newaxis,:]) <= P_line
]
cp.Problem(
    cp.Minimize(cp.vec(C.T)@cp.vec(cp.vstack([(base*p_g)**n for n in range(3)]))),
    constraints
).solve(max_iter=30000)

5840.788888888889

Now let's solve a **demand-side** problem with zero marginal cost, fixed supply, and elastic demand. For this minimal example, we make the follwing assumptions:
- Each load submits a demand curve. The system operator dispatches to maximize total benefit.
- Each load can consume as much power as desired.
- For each load, elasticity is constant. That is, the marginal benefit curve for load $i$ is given by $MB_i(P)=K_iP^{\frac{1}{e_i}}$, where $e_i<-1$ is the elasticity. Note that this formulation insures that marginal benefit is always positive.
- Each load $i$ has a baseline demand $\underline{P}_i$ which it must always consume. Total benefit is measured in excess of the baseline.

For simplicity, suppose each generator supplies its optimal output from the supply-side problem.

To start, we choose $e_1=\dots=e_L=-0.2$. Furthermore, choose $K_i$ by taking the marginal benefit vector to be $MB(P_d^*)=\lambda\mathbf{1}$ where $P_d^*$ is the load vector given by PowerWorld and $\lambda$ is set to $14.62.

In [9]:
e = -0.2
price = 14.620
K = price/(p_d)**(1/e)
p_lower = 0.45*p_d

Solve! To avoid numerical errors, the objective function is in per-unit, so it should not be compared to the supply-side case.

In [10]:
p_d = cp.Variable(len(L))
delta = cp.Variable(N)
constraints = [
    p_d >= p_lower,
    1.8*AG@p_g.value-AL@p_d == -B@delta,
    cp.multiply(B, delta[:,np.newaxis]-delta[np.newaxis,:]) <= P_line
]
cp.Problem(
    cp.Maximize(e/(1+e)*cp.sum(cp.multiply(K,(p_d)**(1/e+1)-(p_lower)**(1/e+1)))),
    constraints
).solve()

355.1334808534154